In [1]:
import torch
from torch.utils.data import DataLoader
from datetime import datetime as dt, timedelta
import pandas as pd
import os
import random
import numpy as np
import torch.nn as nn


# check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
seed = 42  # choose any seed you prefer
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

cuda


In [2]:
# Dataset parameters and Lstm hyperparameters
window_size = 100 # lstm input size

input_window_size = 100

target_window_size = 10 # lstm output size

output_size = 1

hidden_size = 1000

num_layers = 4

dropout = 0.1

In [3]:
class PriceDataset(torch.utils.data.Dataset):
    def __init__(self, item, timespan, start_date_str, end_date_str):
        self.directory = f'C:/Github/PricePrediction/csvfiles/{item}'
        self.item = item
        self.timespan = timespan
        start_date = dt.strptime(start_date_str, '%Y-%m-%d').date()
        end_date = dt.strptime(end_date_str, '%Y-%m-%d').date()
        self.dates = [single_date.strftime("%Y-%m-%d") for single_date in self.daterange(start_date, end_date)]
        self.columns = [1, 4]  # Selecting open and close prices
        self.filenames = self.get_filenames()

    def daterange(self, start_date, end_date):
        for n in range(int((end_date - start_date).days) + 1):
            yield start_date + timedelta(n)

    def get_filenames(self):
        filenames = []
        for date in self.dates:
            filename = f"{self.directory}/{self.item}-{self.timespan}-{date}.csv"
            if os.path.exists(filename):
                filenames.append(filename)
        return filenames

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        df = pd.read_csv(filename, usecols=self.columns, header=None)
        tensor = torch.tensor(df.values, dtype=torch.float)  # Return open and close prices
        return tensor


def sliding_window_percentage(batch):
    windows_percentage = []
    for tensor in batch:
        for i in range(tensor.shape[0] - input_window_size - target_window_size + 1):  # Create windows of size window_size
            window = tensor[i:i+input_window_size+target_window_size]
            pct_change = ((window[:, 1] - window[:, 0]) * 100 / window[:, 0])
            windows_percentage.append(pct_change)
    output_percentage = torch.stack(windows_percentage)

    return output_percentage

def sliding_window_binary(batch):
    windows_binary = []
    for tensor in batch:
        for i in range(tensor.shape[0] - input_window_size - target_window_size+ 1):  # Create windows of size window_size
            window = tensor[i:i+input_window_size+target_window_size]
            binary_change = (window[:, 1] > window[:, 0]).float()  # Calculate the binary change
            windows_binary.append(binary_change)
    output_binary = torch.stack(windows_binary)

    return output_binary

In [4]:
train_dataset = PriceDataset('BTCUSDT', '1m', '2021-03-01', '2023-04-30')
test_dataset = PriceDataset('ETHUSDT', '1m', '2021-03-01', '2023-04-30')


percentage_train_loader = DataLoader(train_dataset, batch_size=1, collate_fn=sliding_window_percentage, shuffle=False, drop_last=True)
percentage_test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_percentage, shuffle=False, drop_last=True)

binary_train_loader = DataLoader(train_dataset, batch_size=1, collate_fn=sliding_window_binary, shuffle=False, drop_last=True)
binary_test_loader = DataLoader(test_dataset, batch_size=1, collate_fn=sliding_window_binary, shuffle=False, drop_last=True)



In [5]:
class PercentagePrediction(nn.Module):
    def __init__(self):
        super(PercentagePrediction, self).__init__()
        self.lstm_pct = nn.LSTM(input_size = input_window_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.fc_pct = nn.Linear(in_features = hidden_size, out_features = output_size, dtype=torch.float)  # output layer for percentage prediction

    def forward(self, x, hidden):
        out, hidden = self.lstm_pct(x, hidden)
        out_pct = self.fc_pct(out)  # output for percentage prediction
        return out_pct, hidden

class BinaryPrediction(nn.Module):
    def __init__(self):
        super(BinaryPrediction, self).__init__()
        self.lstm_binary = nn.LSTM(input_size = input_window_size, hidden_size = hidden_size, num_layers = num_layers, dropout = dropout)
        self.fc_binary = nn.Linear(in_features = hidden_size, out_features = output_size, dtype=torch.float)  # output layer for binary prediction

    def forward(self, x, hidden):
        out, hidden = self.lstm_binary(x, hidden)
        out_binary = self.fc_binary(out)  # output for binary prediction
        return out_binary, hidden


In [6]:
def train(model, train_loader, criterion, optimizer, device):
    model.to(device).train()
    total_batches = len(train_loader)
    loss_sum = float(0)

    # Initialize hidden state
    h0 = torch.zeros(num_layers, hidden_size).to(device)
    c0 = torch.zeros(num_layers, hidden_size).to(device)
    hidden = (h0, c0)

    for i, batch in enumerate(train_loader):
        inputs = batch[:, :input_window_size].to(device)
        targets = batch[:, input_window_size:].to(device)

        optimizer.zero_grad()
        outputs, hidden = model(inputs, hidden)  # Pass the hidden state to the model
        hidden = (hidden[0].detach().to(device), hidden[1].detach().to(device))  # Detach the hidden state from its history

        outputs = outputs.to(device)

        loss = criterion(outputs, targets)
        loss_sum += loss.item()

        loss.backward()
        optimizer.step()

        if torch.isnan(torch.tensor(loss_sum)):
            print(f"{i} batch calculated nan")
            print(inputs.detach().cpu())
            print(outputs.detach().cpu())

            zero_indices = (inputs == 0).nonzero(as_tuple=True)
            one_indices = (inputs == 1).nonzero(as_tuple=True)
            nan_indices = (inputs == torch.nan).nonzero(as_tuple=True)
            
            print("Input Indices of 0s: ", zero_indices)
            print("Input Indices of 1s: ", one_indices)
            print("Input Incides of nans: ", nan_indices)

            # Get the indices of elements in outputs that are 0 or 1
            zero_indices = (outputs == 0).nonzero(as_tuple=True)
            one_indices = (outputs == 1).nonzero(as_tuple=True)
            nan_indices = (outputs == torch.nan).nonzero(as_tuple=True)
            
            print("Output Indices of 0s: ", zero_indices)
            print("Output Indices of 1s: ", one_indices)
            print("Output Incides of nans: ", nan_indices)

            return



        if (i + 1) % 200 == 0:  # Print after every 200 batches
            avg_loss = loss_sum / (i+1)
            print(f"Training progress: [{i + 1}/{total_batches} Batches] \t Avg Loss: {avg_loss:.10f}")


    return loss_sum / total_batches

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    total_batches = len(test_loader)  # Total number of batches

    # Initialize hidden state
    h0 = torch.zeros(num_layers, hidden_size).to(device)
    c0 = torch.zeros(num_layers, hidden_size).to(device)
    hidden = (h0, c0)

    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            inputs = batch[:, :input_window_size].to(device)
            targets = batch[:, input_window_size:].to(device)

            outputs, _ = model(inputs, hidden)  # Get the two outputs
            loss = criterion(outputs, targets)
            total_loss += loss.item()

            if (i + 1) % 200 == 0:  # Print after every 200 batches
                avg_loss = total_loss / (i+1)
                print(f"Testing progress: [{i + 1}/{total_batches} Batches] \t Avg Loss: {avg_loss:.10f}")
    return total_loss / len(test_loader)  # Return average loss

def train_and_evaluate(model, modelname, train_loader, test_loader, criterion, optimizer, epochs, device):
    best_val_loss = float('inf')

    # Try to load existing model and optimizer states
    try:
        # Load the saved models and optimizers
        checkpoint = torch.load(f'models/{str(modelname)}.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Checkpoint loaded. Resuming training from there.")
    except FileNotFoundError:
        print("No checkpoint found. Starting from scratch.")

    for epoch in range(epochs):
        train(model = model, train_loader=train_loader, criterion=criterion, optimizer=optimizer, device=device)
        val_loss = evaluate(model=model, test_loader=test_loader, criterion=criterion, device=device)
        print(f"Epoch {epoch+1}/{epochs} \t {modelname} \t Validation Loss: {val_loss:.10f}")

        # Save the model if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f'models/{str(modelname)}.pth')
            best_val_loss = val_loss


In [16]:
model = PercentagePrediction().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.MSELoss().to(device)

train_and_evaluate(model = model, modelname = 'percentage test model', train_loader=percentage_train_loader, test_loader=percentage_test_loader, criterion=criterion, optimizer=optimizer, device = device, epochs=10)

Checkpoint loaded. Resuming training from there.
Training progress: [200/791 Batches] 	 Avg Loss: 0.0163327558
Training progress: [400/791 Batches] 	 Avg Loss: 0.0126771044
Training progress: [600/791 Batches] 	 Avg Loss: 0.0115010745
Testing progress: [200/791 Batches] 	 Avg Loss: 0.0278223085
Testing progress: [400/791 Batches] 	 Avg Loss: 0.0201047058
Testing progress: [600/791 Batches] 	 Avg Loss: 0.0187871945
Epoch 1/10 	 test model 	 Validation Loss: 0.0160391741
Training progress: [200/791 Batches] 	 Avg Loss: 0.0163277722


KeyboardInterrupt: 

In [7]:
model = BinaryPrediction().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.BCEWithLogitsLoss().to(device)

train_and_evaluate(model = model, modelname = 'binary test model', train_loader=percentage_train_loader, test_loader=percentage_test_loader, criterion=criterion, optimizer=optimizer, device = device, epochs=10)

Checkpoint loaded. Resuming training from there.
Training progress: [200/791 Batches] 	 Avg Loss: 0.0006504046
Training progress: [400/791 Batches] 	 Avg Loss: 0.0006994279
Training progress: [600/791 Batches] 	 Avg Loss: -0.0003572118
Testing progress: [200/791 Batches] 	 Avg Loss: 0.0054584569
Testing progress: [400/791 Batches] 	 Avg Loss: 0.0031136059
Testing progress: [600/791 Batches] 	 Avg Loss: 0.0013784325
Epoch 1/10 	 binary test model 	 Validation Loss: 0.0012619488
Training progress: [200/791 Batches] 	 Avg Loss: 0.0006932730
Training progress: [400/791 Batches] 	 Avg Loss: 0.0007282632
Training progress: [600/791 Batches] 	 Avg Loss: -0.0003606795
Testing progress: [200/791 Batches] 	 Avg Loss: 0.0055666533
Testing progress: [400/791 Batches] 	 Avg Loss: 0.0031701315
Testing progress: [600/791 Batches] 	 Avg Loss: 0.0013927884
Epoch 2/10 	 binary test model 	 Validation Loss: 0.0012751596
Training progress: [200/791 Batches] 	 Avg Loss: 0.0007078977
Training progress: [400

# ------ Printing Tensors -------

In [ ]:
"""
# Size of the Dataset
print(f'Train dataset size: {train_dataset.__getitem__(1)}')
print(f'Test dataset size: {len(train_dataset)}')

# Size of the DataLoader (i.e., number of batches)
print(f'Train dataloader size: {len(binary_train_loader)}')
print(f'Test dataloader size: {len(binary_test_loader)}')

# Size of the tensor output by the Dataset
sample_tensor = train_dataset[0]
print(f'Shape of the tensor output by train_dataset: {sample_tensor.dtype}')
"""
# Size of the tensor output by the DataLoader
for batch in binary_train_loader:
    print(f'Shape of the tensor output by train_loader: {batch.size()}')
    print(batch[3, :])
    break  # we break after the first batch


for batch in binary_train_loader:
    print(f'Shape of the tensor output by train_loader: {batch.dtype}')
    print(batch[3, :])
    break  # we break after the first batch

print(len(binary_train_loader))


# ------ Old Codes ------

In [ ]:
def train(model, train_loader, criterion, optimizer, device, feature):
    model.train()
    total_batches = len(train_loader)
    for i, batch in enumerate(train_loader):
        for j in range(batch.shape[0]): # iterate through the first dimension
            inputs = batch[j, :, feature].unsqueeze(0).to(device) # add an extra dimension to match the model's expected input shape
            percentage_targets = batch[j, 1:, 0].reshape(-1) # Get the percentage change targets
            binary_targets = batch[j, 1:, 1].reshape(-1) # Get the binary change targets

            optimizer.zero_grad()
            percentage_outputs, binary_outputs = model(inputs)
            loss = criterion(percentage_outputs, binary_outputs, percentage_targets, binary_targets)
            loss.backward()
            optimizer.step()

        if (i + 1) % 200 == 0:
            print(f"Training progress: [{i + 1}/{total_batches} Batches]")

In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_batches = len(train_loader)  # Total number of batches
    for i, batch in enumerate(train_loader):  # Use enumerate to get the index (i)
        for j in range(batch.size(0)):
            batch = batch.to(device)
            inputs = batch[j, :]
            percentage_targets = batch[:, 1:, 0]  # Get the percentage change targets
            binary_targets = batch[:, 1:, 1]  # Get the binary change targets

            percentage_targets = percentage_targets.reshape(-1)
            binary_targets = binary_targets.reshape(-1)

            optimizer.zero_grad()
            percentage_outputs, binary_outputs = model(inputs)  # Get the two outputs
            loss = criterion(percentage_outputs, binary_outputs, percentage_targets, binary_targets)
            loss.backward()
            optimizer.step()

        if (i + 1) % 200 == 0:  # Print after every 200 batches
            print(f"Training progress: [{i + 1}/{total_batches} Batches]")

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            inputs = batch[:, :-1, :]
            percentage_targets = batch[:, 1:, 0]  # Get the percentage change targets
            binary_targets = batch[:, 1:, 1]  # Get the binary change targets

            percentage_targets = percentage_targets.reshape(-1)
            binary_targets = binary_targets.reshape(-1)

            percentage_outputs, binary_outputs = model(inputs)  # Get the two outputs
            loss = criterion(percentage_outputs, binary_outputs, percentage_targets, binary_targets)
            total_loss += loss.item()
    return total_loss / len(test_loader)  # Return average loss

def train_and_evaluate(model, modelname, train_loader, test_loader, criterion, optimizer, epochs, device):
    best_val_loss = float('inf')

    # Try to load existing model and optimizer states
    try:
        # Load the saved models and optimizers
        checkpoint = torch.load(f'models/{str(modelname)}.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print("Checkpoint loaded. Resuming training from there.")
    except FileNotFoundError:
        print("No checkpoint found. Starting from scratch.")

    for epoch in range(epochs):
        train(model, train_loader, criterion, optimizer, device)
        val_loss = evaluate(model, test_loader, criterion, device)
        print(f"Epoch {epoch+1}/{epochs} \t {modelname} \t Validation Loss: {val_loss:.10f}")

        # Save the model if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f'models/{str(modelname)}.pth')
            best_val_loss = val_loss


In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    total_batches = len(train_loader)  # Total number of batches
    for i, batch in enumerate(train_loader):  # Use enumerate to get the index (i)
        for j in range(batch.size(0)):
            batch = batch.to(device)
            inputs = batch[j, :]
            targets = batch[j+window_size, :]  # Get the targets

            optimizer.zero_grad()
            outputs = model(inputs)  # Get the two outputs
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        if (i + 1) % 200 == 0:  # Print after every 200 batches
            print(f"Training progress: [{i + 1}/{total_batches} Batches]")

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    total_batches = len(test_loader)  # Total number of batches
    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            batch = batch.to(device)
            inputs = batch[:, :-1, :]
            percentage_targets = batch[:, 1:, 0]  # Get the percentage change targets
            binary_targets = batch[:, 1:, 1]  # Get the binary change targets

            percentage_targets = percentage_targets.reshape(-1)
            binary_targets = binary_targets.reshape(-1)

            percentage_outputs, binary_outputs = model(inputs)  # Get the two outputs
            loss = criterion(percentage_outputs, binary_outputs, percentage_targets, binary_targets)
            total_loss += loss.item()

            if (i + 1) % 200 == 0:  # Print after every 200 batches
                print(f"Testing progress: [{i + 1}/{total_batches} Batches]")
    return total_loss / len(test_loader)  # Return average loss

def train_and_evaluate(model, modelname, train_loader, test_loader, criterion, optimizer, epochs, device):
    best_val_loss = float('inf')

    for epoch in range(epochs):
        train(model, train_loader, criterion, optimizer, device)
        val_loss = evaluate(model, test_loader, criterion, device)
        print(f"Epoch {epoch+1}/{epochs} \t {modelname} \t Validation Loss: {val_loss:.10f}")

        # Save the model if the validation loss is the best we've seen so far.
        if val_loss < best_val_loss:
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f'models/{str(modelname)}.pth')
            best_val_loss = val_loss

In [ ]:
class CustomCriterion(nn.Module):
    def __init__(self, weights=(1.0, 1.0)):
        super().__init__()
        self.loss_fn_pct = nn.MSELoss()
        self.loss_fn_binary = nn.BCEWithLogitsLoss()
        self.weights = weights

    def forward(self, percentage_outputs, binary_outputs, percentage_targets, binary_targets):
        loss_pct = self.loss_fn_pct(percentage_outputs, percentage_targets)
        loss_binary = self.loss_fn_binary(binary_outputs, binary_targets)
        return self.weights[0] * loss_pct + self.weights[1] * loss_binary


In [ ]:
model = PriceChangePrediction().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = CustomCriterion().to(device)

try: 
    # Load the saved models and optimizers
    checkpoint = torch.load('C:/Github/PricePrediction/docker/models/combined_model.pth')

    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

except FileNotFoundError:
    print("No checkpoint found. Starting from scratch.")
    hidden = (torch.zeros(num_layers, hidden_size).to(device), torch.zeros(num_layers, hidden_size).to(device))  # need to write code for initializing hidden state tensor

epochs = 1  # or any other number you prefer

train_and_evaluate(model, 'combined_model', train_loader, test_loader, criterion, optimizer, epochs, device)
